6.7.2 读取数据集

In [10]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import zipfile
import sys
sys.path.append('..')
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
def load_data_jay_lyrics():
    """加载周杰伦歌词数据集"""
    with zipfile.ZipFile('../Datasets/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

6.7.3 从零开始实现GRU
    6.7.3.1 初始化模型参数

In [12]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('Will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    
    def _three():
        return (
            _one((num_inputs, num_hiddens)),
            _one((num_hiddens, num_hiddens)),
            torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True)
        )
    
    W_xz, W_hz, b_z = _three()
    W_xr, W_hr, b_r = _three()
    W_xh, W_hh, b_h = _three()

    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])


Will use cuda


6.7.3.2 定义模型

In [13]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [14]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

6.7.3.3 训练模型并创作歌词

In [15]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens, vocab_size,
                          device, corpus_indices, idx_to_char, char_to_idx, False,
                          num_epochs, num_steps, lr, clipping_theta, batch_size,pred_period, pred_len, prefixes)

epoch 40, perplexity 148.002437, time 0.40 sec
 - 分开 我想你的让我 你想我想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
 - 不分开 我想你的让我 你想我想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
epoch 80, perplexity 30.565626, time 0.37 sec
 - 分开 我想要你的微笑 我想你这样 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想
 - 不分开  没有你在我有 有你的话笑 我想要你的微笑 我想你这样 我不要再想 我不要再想 我不要再想 我不要
epoch 120, perplexity 5.716406, time 0.36 sec
 - 分开 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样
 - 不分开我 你不再 不知我 不要我 说你怎么对我 别发抖 快给我抬起头 有话去对医药箱说 别怪我 别怪我 说
epoch 160, perplexity 1.773227, time 0.39 sec
 - 分开 这小的梦猫 喜欢它 瞎念常 有属于  有漠么么对着我有攻  说说了飞 我有多烦恼 我想要你的微笑每
 - 不分开不远 这小我的泪是就 说通 在又再考倒我 说散 你想很久了吧? 败给你的黑色幽默 不想太多 我想一定


In [16]:
lr = 1e-2
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.017083, time 0.05 sec
 - 分开的玩笑 想通 却又再考倒我 说散 你想很久了吧? 败给你的黑色幽默 不想太多 我想一定是我听错弄错搞
 - 不分开始 担心今天的你过得好不好 整个画面是你 想你想的睡不著 嘴嘟嘟那可爱的模样 还有在你身上香香的味道
epoch 80, perplexity 1.011029, time 0.06 sec
 - 分开的玩笑 想通 却又再考倒我 说散 你想很久了吧? 败给你的黑色幽默 不想太多 我想一定是我听错弄错搞
 - 不分开球 已经不是你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 你爱过我的证据 让晶莹的泪
epoch 120, perplexity 1.007677, time 0.05 sec
 - 分开的手不放开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样没担忧 唱着歌 一
 - 不分开 如果我遇见你是一场悲剧 我想我这辈子注定一个人演戏 最后再一个人慢慢的回忆 没有了过去 我将往事抽
epoch 160, perplexity 1.008421, time 0.05 sec
 - 分开 我想就这样牵着你的手不放开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样
 - 不分开  有知不觉  已经不是我 上海一九四三 泛黄的春联还残无力的躺在干枯的河 在等待雨季来临变沼泽 灰
